# PyCity Schools Analysis

- - [ ] There is a significant decrease in the over all passing % for larger schools compared to smaller schools. Larger schools are almost have the over all passing percentage compared to medium and small schools. These larger schools are mainly district schools. These district schools often have the higher spending ranges per student compared to the Charter schools. Although they may have higher spending per student the way the resources are allocated may not be similar to those of Charter schools who are able to reach these higher passing percentages. The data shows the changes in student math and reading scores based on school size, budget, and type of school. 

  
---

In [36]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# Global float format
pd.options.display.float_format = '{:.3f}%'.format

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["School ID"].unique())
school_count

15

In [3]:
# Calculate the total number of students
student_count = len(school_data_complete["Student ID"].unique())
student_count

39170

In [4]:
# Calculate the total budget
total_budget = school_data["budget"].sum()
total_budget

24649428

In [5]:
# Calculate the average (mean) math score
average_math_score = student_data["math_score"].mean()
average_math_score

78.98537145774827

In [6]:
# Calculate the average (mean) reading score
average_reading_score = student_data["reading_score"].mean()
average_reading_score

81.87784018381414

In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [8]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count /float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [9]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = [{"Total Schools": school_count, 
                     "Total Students": student_count, 
                     "Total Budget": total_budget, 
                     "Average Math Score": round(average_math_score,2), 
                     "Average Reading Score": round(average_reading_score,2), 
                     "% Passing Math": round(passing_math_percentage,2), 
                     "% Passing Reading": round(passing_reading_percentage,2), 
                     "% Overall Passing": round(overall_passing_rate,2)}]
district_summary_table = pd.DataFrame(district_summary)

# Formatting
district_summary_table["Total Schools"] = district_summary_table["Total Schools"].map("{:,}".format)
district_summary_table["Total Students"] = district_summary_table["Total Students"].map("{:,}".format)
district_summary_table["Total Budget"] = district_summary_table["Total Budget"].map("${:,.2f}".format)
district_summary_table["Average Math Score"] = district_summary_table["Average Math Score"].map("{:,}".format)
district_summary_table["Average Reading Score"] = district_summary_table["Average Reading Score"].map("{:,}".format)
district_summary_table["% Passing Math"] = district_summary_table["% Passing Math"].map("{:,.2f}%".format)
district_summary_table["% Passing Reading"] = district_summary_table["% Passing Reading"].map("{:,.2f}%".format)
district_summary_table["% Overall Passing"] = district_summary_table["% Overall Passing"].map("{:,.2f}%".format)

# Display the DataFrame
district_summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

In [11]:
# Use the code provided to select all of the school types
school_types = school_data.set_index(["school_name"])["type"]
school_types.head()

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Name: type, dtype: object

In [12]:
# Calculate the total student count per school
per_school_counts = school_data.set_index(["school_name"])["size"]
per_school_counts.head()

school_name
Huang High School        2917
Figueroa High School     2949
Shelton High School      1761
Hernandez High School    4635
Griffin High School      1468
Name: size, dtype: int64

In [13]:
# Calculate the total school budget and per capita spending per school
per_school_budget = school_data_complete.groupby(["school_name"])["budget"].max()
print(per_school_budget.head())

per_school_capita = (per_school_budget / per_school_counts).astype('int')
per_school_capita.head()

school_name
Bailey High School      3124928
Cabrera High School     1081356
Figueroa High School    1884411
Ford High School        1763916
Griffin High School      917500
Name: budget, dtype: int64


school_name
Bailey High School      628
Cabrera High School     582
Figueroa High School    639
Ford High School        644
Griffin High School     625
dtype: int64

In [14]:
# Calculate the average test scores per school
per_school_math = school_data_complete.groupby(["school_name"])["math_score"].mean()
print(per_school_math.head())
per_school_reading = school_data_complete.groupby(["school_name"])["reading_score"].mean()
per_school_reading.head()

school_name
Bailey High School     77.048%
Cabrera High School    83.062%
Figueroa High School   76.712%
Ford High School       77.103%
Griffin High School    83.351%
Name: math_score, dtype: float64


school_name
Bailey High School     81.034%
Cabrera High School    83.976%
Figueroa High School   81.158%
Ford High School       80.746%
Griffin High School    83.817%
Name: reading_score, dtype: float64

In [15]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]
school_students_passing_math = students_passing_math.groupby(["school_name"]).size()
school_students_passing_math.head()

school_name
Bailey High School      3318
Cabrera High School     1749
Figueroa High School    1946
Ford High School        1871
Griffin High School     1371
dtype: int64

In [16]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]
school_students_passing_reading = students_passing_reading.groupby(["school_name"]).size()
school_students_passing_reading.head()

school_name
Bailey High School      4077
Cabrera High School     1803
Figueroa High School    2381
Ford High School        2172
Griffin High School     1426
dtype: int64

In [17]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()
school_students_passing_math_and_reading.head()

school_name
Bailey High School      2719
Cabrera High School     1697
Figueroa High School    1569
Ford High School        1487
Griffin High School     1330
dtype: int64

In [18]:
# Use the provided code to calculate the passing rates
per_school_passing_math = students_passing_math.groupby(["school_name"])
per_school_passing_math = school_students_passing_math / per_school_counts * 100
print("Passing Math Rate:")
print(per_school_passing_math.head())

print("Passing Reading Rate:")
per_school_passing_reading = students_passing_reading.groupby(["school_name"])
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
print(per_school_passing_reading.head())

overall_passing_rate = students_passing_math_and_reading.groupby(["school_name"])
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100
print("Overall Passing Rate:")
print(overall_passing_rate.head())

Passing Math Rate:
school_name
Bailey High School     66.680%
Cabrera High School    94.133%
Figueroa High School   65.988%
Ford High School       68.310%
Griffin High School    93.392%
dtype: float64
Passing Reading Rate:
school_name
Bailey High School     81.933%
Cabrera High School    97.040%
Figueroa High School   80.739%
Ford High School       79.299%
Griffin High School    97.139%
dtype: float64
Overall Passing Rate:
school_name
Bailey High School     54.642%
Cabrera High School    91.335%
Figueroa High School   53.204%
Ford High School       54.290%
Griffin High School    90.599%
dtype: float64


In [19]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame ({"School Type" : school_types, 
                                  "Total Students" : per_school_counts,
                                  "Total School Budget" : per_school_budget,
                                  "Per Student Budget" : per_school_capita,
                                  "Average Math Score" : per_school_math,
                                  "Average Reading Score" : per_school_reading,
                                  "% Passing Math" : per_school_passing_math,
                                  "% Passing Reading" : per_school_passing_reading,
                                   "% Overall Passing": overall_passing_rate})

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628,77.048%,81.034%,66.680%,81.933%,54.642%
Cabrera High School,Charter,1858,1081356,582,83.062%,83.976%,94.133%,97.040%,91.335%
Figueroa High School,District,2949,1884411,639,76.712%,81.158%,65.988%,80.739%,53.204%
Ford High School,District,2739,1763916,644,77.103%,80.746%,68.310%,79.299%,54.290%
Griffin High School,Charter,1468,917500,625,83.351%,83.817%,93.392%,97.139%,90.599%
Hernandez High School,District,4635,3022020,652,77.290%,80.934%,66.753%,80.863%,53.528%
Holden High School,Charter,427,248087,581,83.803%,83.815%,92.506%,96.253%,89.227%
Huang High School,District,2917,1910635,655,76.629%,81.183%,65.684%,81.316%,53.514%
Johnson High School,District,4761,3094650,650,77.072%,80.966%,66.058%,81.222%,53.539%


## Highest-Performing Schools (by % Overall Passing)

In [20]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("% Overall Passing", ascending = False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582,83.062%,83.976%,94.133%,97.040%,91.335%
Thomas High School,Charter,1635,1043130,638,83.418%,83.849%,93.272%,97.309%,90.948%
Griffin High School,Charter,1468,917500,625,83.351%,83.817%,93.392%,97.139%,90.599%
Wilson High School,Charter,2283,1319574,578,83.274%,83.989%,93.868%,96.540%,90.583%
Pena High School,Charter,962,585858,609,83.840%,84.045%,94.595%,95.946%,90.541%


## Bottom Performing Schools (By % Overall Passing)

In [21]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values("% Overall Passing")
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637,76.843%,80.745%,66.367%,80.220%,52.988%
Figueroa High School,District,2949,1884411,639,76.712%,81.158%,65.988%,80.739%,53.204%
Huang High School,District,2917,1910635,655,76.629%,81.183%,65.684%,81.316%,53.514%
Hernandez High School,District,4635,3022020,652,77.290%,80.934%,66.753%,80.863%,53.528%
Johnson High School,District,4761,3094650,650,77.072%,80.966%,66.058%,81.222%,53.539%


## Math Scores by Grade

In [22]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grader_math_scores = ninth_graders.groupby(["school_name"])["math_score"].mean()
tenth_grader_math_scores = tenth_graders.groupby(["school_name"])["math_score"].mean()
eleventh_grader_math_scores = eleventh_graders.groupby(["school_name"])["math_score"].mean()
twelfth_grader_math_scores = twelfth_graders.groupby(["school_name"])["math_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({"9th": ninth_grader_math_scores,
                                    "10th": tenth_grader_math_scores,
                                    "11th": eleventh_grader_math_scores,
                                    "12th": twelfth_grader_math_scores}) 

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.084%,76.997%,77.516%,76.492%
Cabrera High School,83.095%,83.155%,82.766%,83.277%
Figueroa High School,76.403%,76.540%,76.884%,77.151%
Ford High School,77.361%,77.672%,76.918%,76.180%
Griffin High School,82.044%,84.229%,83.842%,83.356%
Hernandez High School,77.438%,77.337%,77.136%,77.187%
Holden High School,83.787%,83.430%,85.000%,82.855%
Huang High School,77.027%,75.909%,76.447%,77.226%
Johnson High School,77.188%,76.691%,77.492%,76.863%
Pena High School,83.625%,83.372%,84.328%,84.122%


## Reading Score by Grade 

In [23]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grader_reading_scores = ninth_graders.groupby(["school_name"])["reading_score"].mean()
tenth_grader_reading_scores = tenth_graders.groupby(["school_name"])["reading_score"].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby(["school_name"])["reading_score"].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby(["school_name"])["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({"9th": ninth_grader_reading_scores,
                                    "10th": tenth_grader_reading_scores,
                                    "11th": eleventh_grader_reading_scores,
                                    "12th": twelfth_grader_reading_scores}) 

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None




# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303%,80.907%,80.946%,80.912%
Cabrera High School,83.676%,84.253%,83.788%,84.288%
Figueroa High School,81.199%,81.409%,80.640%,81.385%
Ford High School,80.633%,81.263%,80.404%,80.662%
Griffin High School,83.369%,83.707%,84.288%,84.014%
Hernandez High School,80.867%,80.660%,81.396%,80.857%
Holden High School,83.677%,83.325%,83.816%,84.699%
Huang High School,81.290%,81.512%,81.417%,80.306%
Johnson High School,81.261%,80.773%,80.616%,81.228%
Pena High School,83.807%,83.612%,84.336%,84.591%


## Scores by School Spending

In [24]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [25]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()


In [26]:
# Use `pd.cut` to categorize spending based on the bins.

school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, 
                                                            labels=labels)

school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928,628,77.048%,81.034%,66.680%,81.933%,54.642%,$585-630
Cabrera High School,Charter,1858,1081356,582,83.062%,83.976%,94.133%,97.040%,91.335%,<$585
Figueroa High School,District,2949,1884411,639,76.712%,81.158%,65.988%,80.739%,53.204%,$630-645
Ford High School,District,2739,1763916,644,77.103%,80.746%,68.310%,79.299%,54.290%,$630-645
Griffin High School,Charter,1468,917500,625,83.351%,83.817%,93.392%,97.139%,90.599%,$585-630
Hernandez High School,District,4635,3022020,652,77.290%,80.934%,66.753%,80.863%,53.528%,$645-680
Holden High School,Charter,427,248087,581,83.803%,83.815%,92.506%,96.253%,89.227%,<$585
Huang High School,District,2917,1910635,655,76.629%,81.183%,65.684%,81.316%,53.514%,$645-680
Johnson High School,District,4761,3094650,650,77.072%,80.966%,66.058%,81.222%,53.539%,$645-680


In [27]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [28]:
school_spending_df.dtypes

School Type                        object
Total Students                      int64
Total School Budget                 int64
Per Student Budget                  int64
Average Math Score                float64
Average Reading Score             float64
% Passing Math                    float64
% Passing Reading                 float64
% Overall Passing                 float64
Spending Ranges (Per Student)    category
dtype: object

In [29]:
# Assemble into DataFrame
spending_summary = school_spending_df = pd.DataFrame({"Average Math Score": spending_math_scores,
                                                     "Average Reading Score": spending_reading_scores,
                                                     "% Passing Math": spending_passing_math,
                                                     "% Passing Reading": spending_passing_reading,
                                                     "% Overall Passing": overall_passing_spending})

spending_summary["Average Math Score"] = spending_summary["Average Math Score"].map("{:.2f}".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:.2f}".format)

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.460%,96.611%,90.369%
$585-630,81.90,83.16,87.134%,92.718%,81.419%
$630-645,78.52,81.62,73.484%,84.392%,62.858%
$645-680,77.00,81.03,66.165%,81.134%,53.527%


## Scores by School Size

In [30]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [31]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.


per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels = labels) 

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928,628,77.048%,81.034%,66.680%,81.933%,54.642%,Large (2000-5000)
Cabrera High School,Charter,1858,1081356,582,83.062%,83.976%,94.133%,97.040%,91.335%,Medium (1000-2000)
Figueroa High School,District,2949,1884411,639,76.712%,81.158%,65.988%,80.739%,53.204%,Large (2000-5000)
Ford High School,District,2739,1763916,644,77.103%,80.746%,68.310%,79.299%,54.290%,Large (2000-5000)
Griffin High School,Charter,1468,917500,625,83.351%,83.817%,93.392%,97.139%,90.599%,Medium (1000-2000)
Hernandez High School,District,4635,3022020,652,77.290%,80.934%,66.753%,80.863%,53.528%,Large (2000-5000)
Holden High School,Charter,427,248087,581,83.803%,83.815%,92.506%,96.253%,89.227%,Small (<1000)
Huang High School,District,2917,1910635,655,76.629%,81.183%,65.684%,81.316%,53.514%,Large (2000-5000)
Johnson High School,District,4761,3094650,650,77.072%,80.966%,66.058%,81.222%,53.539%,Large (2000-5000)


In [32]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [33]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score" : size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading" : size_passing_reading, 
    "% Overall Pass" : size_overall_passing
})


size_summary["Average Math Score"] = size_summary["Average Math Score"].map("{:.2f}".format)
size_summary["Average Reading Score"] = size_summary["Average Reading Score"].map("{:.2f}".format)
size_summary["% Passing Math"] = size_summary["% Passing Math"].map("{:.0f}%".format)
size_summary["% Passing Reading"] = size_summary["% Passing Reading"].map("{:.0f}%".format)
size_summary["% Overall Pass"] = size_summary["% Overall Pass"].map("{:.0f}%".format)


# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass
School Size,,,,,
Small (<1000),83.82,83.93,94%,96%,90%
Medium (1000-2000),83.37,83.86,94%,97%,91%
Large (2000-5000),77.75,81.34,70%,83%,58%


## Scores by School Type

In [34]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [35]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score": average_math_score_by_type,
    "Average Reading Score" : average_reading_score_by_type,
    "% Passing Math": average_percent_passing_math_by_type,
    "% Passing Reading" : average_percent_passing_reading_by_type, 
    "% Overall Pass" : average_percent_overall_passing_by_type
})

# Display results
type_summary["Average Math Score"] = type_summary["Average Math Score"].map("{:.1f}".format)
type_summary["Average Reading Score"] = type_summary["Average Reading Score"].map("{:.1f}".format)
type_summary["% Passing Math"] = type_summary["% Passing Math"].map("{:.0f}%".format)
type_summary["% Passing Reading"] = type_summary["% Passing Reading"].map("{:.0f}%".format)
type_summary["% Overall Pass"] = type_summary["% Overall Pass"].map("{:.0f}%".format)

type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass
School Type,,,,,
Charter,83.5,83.9,94%,97%,90%
District,77.0,81.0,67%,81%,54%
